In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py

In [ ]:
data = pd.read_csv('inputs/Cajamar.csv')
data.columns

In [ ]:
def create_train_set(df: pd.DataFrame, cluster: int) -> pd.DataFrame:
    data = df[df['Cluster']==cluster]
    train_dataset= pd.DataFrame()
    train_dataset['ds'] = pd.to_datetime(data["time"])
    train_dataset['y']=data[['y_0']]
    return train_dataset

def train_prophet(train_dataset):
    prophet_basic = Prophet(daily_seasonality=True)
    prophet_basic.fit(train_dataset)
    return prophet_basic

def make_pred(prophet_basic, period):
    future = prophet_basic.make_future_dataframe(periods=period)
    forecast = prophet_basic.predict(future)
    return forecast

## Entrenamiento del modelo de Prophet con los tres clusters    

In [ ]:
train_dataset = create_train_set(data, 1)
prophet_fit = train_prophet(train_dataset)
forecast = make_pred(prophet_fit, 14)

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly
plot_plotly(prophet_fit, forecast)

In [ ]:
plot_components_plotly(prophet_fit, forecast)

In [ ]:
## Diagnostics: Cross validation para ver cual va siendo el error de training
from prophet.diagnostics import cross_validation
df_cv = cross_validation(prophet_fit, initial='256 days', period='1 days', horizon = '15 days')

In [ ]:
df_cv.head()

In [ ]:
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='rmse')

In [ ]:
import itertools
import numpy as np
import pandas as pd

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

train_dataset = create_train_set(data, 1)
cutoffs = pd.to_datetime(['2019-10-31', '2019-11-30', '2019-12-31'], dayfirst=True)

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(train_dataset)  # Fit model with given params
    df_cv = cross_validation(m, cutoffs=cutoffs, horizon='15 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)